## Imports


In [1]:
import os
import json
import pandas as pd
import psycopg2
from psycopg2 import OperationalError
from psycopg2 import ProgrammingError

## Data Connection


In [2]:
# read user file
with open("package/account/user.json", "r") as read_file:
    user = json.load(read_file)

In [3]:
# make discord name
discord_username_full = user["username"] + "#" + str(user["discriminator"])

In [4]:
# function to connect to postgresql db
def create_connection(db_name, db_user, db_password, db_host, db_port):
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        connection = None
        print(f"The error '{e}' occurred")
        print("Error Type:", type(e))

    return connection

In [5]:
def read_secrets() -> dict:
    filename = os.path.join("secrets.json")
    try:
        with open(filename, mode="r") as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return {}

In [6]:
secrets = read_secrets()


In [7]:
# set up database connection
con = create_connection(
    secrets["db_name"],
    secrets["db_user"],
    secrets["db_password"],
    secrets["db_host"],
    secrets["db_port"],
)


Connection to PostgreSQL DB successful


In [8]:
cur = con.cursor()


In [9]:
# function to select data, returns a dataframe
def select(query):
    data_df = None
    try:
        cur.execute(query)
        col_names = [desc[0] for desc in cur.description]
        data_df = pd.DataFrame(data=cur.fetchall(), columns=col_names)
    except ProgrammingError as e:
        con.rollback()
        print(f"The error '{e}' occurred")
        print("Error Type:", type(e))

    return data_df

## Import Data


In [10]:
select_query = """
SELECT *
FROM discord_data.dim_channel
"""

df_dim_channel = select(select_query)


In [11]:
for column in df_dim_channel:
    if df_dim_channel[column].dtype == 'float64':
        df_dim_channel[column] = pd.to_numeric(
            df_dim_channel[column], downcast='float')
    if df_dim_channel[column].dtype == 'int64':
        df_dim_channel[column] = pd.to_numeric(
            df_dim_channel[column], downcast='integer')

In [12]:
select_query = """
SELECT *
FROM discord_data.dim_channel_type
"""

df_dim_channel_type = select(select_query)


In [13]:
for column in df_dim_channel_type:
    if df_dim_channel_type[column].dtype == 'float64':
        df_dim_channel_type[column] = pd.to_numeric(
            df_dim_channel_type[column], downcast='float')
    if df_dim_channel_type[column].dtype == 'int64':
        df_dim_channel_type[column] = pd.to_numeric(
            df_dim_channel_type[column], downcast='integer')

In [14]:
select_query = """
SELECT *
FROM discord_data.dim_server
"""

df_dim_server = select(select_query)


In [15]:
for column in df_dim_server:
    if df_dim_server[column].dtype == 'float64':
        df_dim_server[column] = pd.to_numeric(
            df_dim_server[column], downcast='float')
    if df_dim_server[column].dtype == 'int64':
        df_dim_server[column] = pd.to_numeric(
            df_dim_server[column], downcast='integer')

In [16]:
select_query = """
SELECT *
FROM discord_data.fact_messages
"""

df_fact_messages = select(select_query)


In [17]:
for column in df_fact_messages:
    if df_fact_messages[column].dtype == 'float64':
        df_fact_messages[column] = pd.to_numeric(
            df_fact_messages[column], downcast='float')
    if df_fact_messages[column].dtype == 'int64':
        df_fact_messages[column] = pd.to_numeric(
            df_fact_messages[column], downcast='integer')

In [18]:
select_query = """
SELECT *
FROM discord_data.dim_channel dc
LEFT JOIN discord_data.dim_channeL_type dct
on dc.channel_type_key = dct.channel_type_key
LEFT JOIN discord_data.dim_server ds
on dc.server_id = ds.server_id
LEFT JOIN discord_data.fact_messages fm
ON fm.channel_id = dc.channel_id
"""

df = select(select_query)


In [19]:
df.head()

,channel_id,channel_name,channel_type_key,server_id,channel_type_key,channel_type,server_id,server_name,message_id,message_timestamp,contents,attachment_link,channel_id
0,1019163001686855720,mochi fan club,3,NaN,3,Group DM,NaN,None,1.054978e+18,2022-12-21 04:25:35.105000+00:00,im not familiar with it but all the best,None,1.019163e+18
1,1019163001686855720,mochi fan club,3,NaN,3,Group DM,NaN,None,1.054978e+18,2022-12-21 04:24:21.416000+00:00,oh no,None,1.019163e+18
2,1019163001686855720,mochi fan club,3,NaN,3,Group DM,NaN,None,1.054977e+18,2022-12-21 04:23:46.918000+00:00,hi,None,1.019163e+18
3,1019163001686855720,mochi fan club,3,NaN,3,Group DM,NaN,None,1.054977e+18,2022-12-21 04:23:13.260000+00:00,<a:r_rage:832036415436488765>,None,1.019163e+18
4,1019163001686855720,mochi fan club,3,NaN,3,Group DM,NaN,None,1.054977e+18,2022-12-21 04:23:11.610000+00:00,'ppl don wan u or u don wan ppl?',None,1.019163e+18


In [20]:
# read from csv (due to it being semi structured data)
# may transition to a NoSQL database in the future
# store each list as its own csv file
# length = 6
# df_fact_analytics = pd.DataFrame()
# for i in range(1, length+1, 1):
#     file_name = 'fact_analytics' + str(i) + '.csv'
#     df_temp = pd.read_csv(file_name)
#     df_fact_analytics = pd.concat([df_fact_analytics, df_temp])
#     print(df_fact_analytics.shape)

# not running due to memory limitations


In [21]:
# df_fact_analytics.head()

In [22]:
# read from csv (due to it being semi structured data)
# may transition to a NoSQL database in the future
df_fact_modeling = pd.read_csv("fact_modeling.csv")

C:\Users\Dante\AppData\Local\Temp\ipykernel_6516\1362443282.py:3: DtypeWarning: Columns (10,11,13,14,21,27,28,34,36,40,41,43,44,45,46,47,50,51,52,59,60,63,65,67,68,69,70,71,82,85,89,90,91,92,93,94,95,96,97,98,99,100,101,103,104,105,110,113,114,115,116,124,125,128,129,130,132,133,134,135,136,137,144,145,148,149,150,152,153,154,159,160,161,162,170,171,172,173,174,176,178,181,182,183,184,185,189,191,192,194,195,197,199,200,201,202,203,206,207,209,212,214,215,216,217,218,219,228,229,230,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,250,252,253,254,255,257,258,259,260,261,262,265,266,267,268,269,270,271,272,273,277,278,321,322,324,325,326,340,342,347,348,349,350,351,361,362,363,435,436,437,439,474,497,498,504,505,506,510,512,513,514,515,518,520,521,522,523,528,529,530,531,532,533,534,535,536,537,541,542,543,545,546,548,550,557,562,563,564,565,566,567,568,569,570,571,572,573,579,580,581,582,585,586,589,590,591,592,593,594,595,596,597,598,603,604,606,608,612,614,616,617,620,621,

In [23]:
df_fact_modeling.head()

,event_type,event_id,event_source,user_id,domain,freight_hostname,ip,day,chosen_locale,detected_locale,...,num_guilds_recommended,num_guilds_popular,recommended_guild_ids,category_id,static_route,quantity,payment_modal_version,feed_item_type,active_events_shown,upcoming_events_shown
0,send_message,AQEEeMaeN7LgGi+m6iTXOLDyUgAdP/4=,api,342346882800025600,Modeling,discord-api-7fc4cb59bd-6scxz,103.253.105.0,1855,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,send_message,AQEElawFMUYV3IILbLXWUN3aWgAzKRI=,api,342346882800025600,Modeling,api-prd-main-sj45,14.192.208.0,1393,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,send_message,AQEEn79YxgtyTW7p/U/KFZVVzgE+KL8=,api,342346882800025600,Modeling,api-prd-main-p15l,14.192.214.0,1271,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,send_message,AQEEnywJE8wbsdRf4V5DtWgU2wDS3hM=,api,342346882800025600,Modeling,api-prd-main-4krc,14.192.214.0,1267,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,send_message,AQEExt4rlm/9JkoZb0Nl2xTIKQAITZI=,api,342346882800025600,Modeling,api-prd-main-d693,14.192.217.0,1473,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# read from csv (due to it being semi structured data)
# may transition to a NoSQL database in the future
df_fact_reporting = pd.read_csv("fact_reporting.csv")

C:\Users\Dante\AppData\Local\Temp\ipykernel_6516\473704480.py:3: DtypeWarning: Columns (36,40,93,109,111,112,113,114,116,117,119,122,123,125,126,127,128,129,130,131,132,133,134,136,138,139,140,141,142,143,144,145,148,149,150,151,152,153,154,156,157,165,167,168,169,172,173,174,175,176,177,178,179,183,184,185,187,188,189,190,197,198,199,200,202,203,204,205,220,267,275,276,277,278,279,283,287,288,290,291,293,295,306,307,316,317,328,329,333,343,344,345,347,348,349,350,351,353,354,357,358,359,360,361,362,363,364,366,367,368,369,370,371,372,373,374,377,378,379,384,385,386,387,388,389,390,394,396,397,400,401,402,403,404,405,409,410,411,412,413,416,419,420,423,427,433,435,436,438,439,440,441,442,443,444,447,448,449,450,451,452,453,454,455,456,457,460,462,463,464,465,468,469,472,473,476,477,478,479,480,482,493,494,495,496,500,502,515,518,521,523,529,530,531,532,533,534,535,536,537,538,539,541,542,544,545,546,547,548,549,552,555,556,558,560,562,563,564,566,567,568,570,571,573,574,577,578,580,582

In [25]:
df_fact_reporting.head()

,event_type,event_id,event_source,user_id,domain,freight_hostname,ip,day,chosen_locale,detected_locale,...,settlement_fees,settlement_tax,login_source,removal_type,num_channels_highlighted,full,instant_invite,notifications_in_app_enabled,is_premium,impression_group
0,send_message,AQECVcvXZPjotGaE6cJJXVly+QAmzjg=,api,342346882800025600,Reporting,discord-api-8668c6c64f-76szd,103.253.105.0,1786,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,send_message,AQEC1nfTIOe7jEZUm9Zvq0MU5wCYp+k=,api,342346882800025600,Reporting,api-prd-main-2rvc,14.192.211.0,1456,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,send_message,AQEC1Q07SE3AoHYjZp0YIl3s+wA6+Xs=,api,342346882800025600,Reporting,discord-api-6b5587cdd-kg684,103.253.105.0,1884,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,send_message,AQECWhxO4OlYdt9LtC5A2+f5iQGnIfg=,api,342346882800025600,Reporting,discord-api-768499f95b-xgk4k,103.253.105.0,1775,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,send_message,AQEC9gPhJ16/o3QM+VEXoEvNPgAUSnU=,api,342346882800025600,Reporting,api-prd-main-mvh0,14.192.211.0,1450,en-GB,en-GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# read from csv (due to it being semi structured data)
# may transition to a NoSQL database in the future
df_fact_tns = pd.read_csv("fact_tns.csv")

C:\Users\Dante\AppData\Local\Temp\ipykernel_6516\1577899063.py:3: DtypeWarning: Columns (12,24,44,45,49,50,68,69,72,74,81,83,84,86,89,90,91,103,107,131,135,137,138,139,143,144,145,146,148,151,153,154,155,156,157,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,176,177,178,179,181,182,183,184,239,240,242,243,244,245,246,249,250,251,261,262,264,267,268,269,270,271,272,273,274,275,276,277,278,279,281,282,283,287,288,289,296,297,298,301,303,304,305,306,315,317,318,319,321,322,323,324,325,326,327,328,329,330,331,332,333,334,338,340,341,342,343,345,346,347,348,349,371,372,373,379,380,411,419,421,422,423,429,431,440,441,442,443,444,448,459,461,462,463,473,474,475,548,549,555,597,599,601,602,603,607,608,611,612,613,615,616,619,620,621,627,628,656,659,660,661,662,663,664,665,666,667,668,669,670,671,672,674,675,676,682,685,687,689,690,693,694,695,707,708,711,716,717,718,719,721,723,724,725,726,727,728,731,742,743,747,752,753,760,761,762,763,764,765,766,767,769,780,781,782,783,784,

In [27]:
df_fact_tns.head()

,event_type,event_id,event_source,user_id,domain,client_uuid,freight_hostname,freight_id,ip,day,...,payment_source_type,is_default,card_brand,card_expiration_date,step_duration_ms,flow_duration_ms,desktop_ip,desktop_country_code,country_match,abort_with_captcha
0,start_listening,AQMFhMhBL8SZO0YUjhdbm/2E1wAAAxs=,client,342346882800025600,Tns,AABCE61CwAQVTLxrUXhpHoEBAAAbAwAA,analytics-ingest-prd-mn2k,153Y7cSI37s9FO4WAM11Qg==,103.253.105.84,1764,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,start_listening,AQMFug5sBO3k0MAbSRDNOQJ+7QAAA2k=,client,342346882800025600,Tns,AABCE61CwAQvEvyLgCKXk4EBAABpAwAA,analytics-ingest-prd-ndw8,ej4kMneeDJexkPoWpgLd3A==,103.253.105.84,1787,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,start_listening,AQMF+ElFOpiHmf84royFTrxiWgAAAz0=,client,342346882800025600,Tns,AABCE61CwARu5QGi2id+fIEBAAA9AwAA,analytics-ingest-prd-q0pz,HiRxikGPr_LjE-4W5_eiyQ==,103.253.105.84,1782,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,start_listening,AQMF+ElFOpiHmf84royFTrxiWgAAAc0=,client,342346882800025600,Tns,AABCE61CwARu5QGi2id+fIEBAADNAQAA,analytics-ingest-prd-d494,9yPZ_VQ0xDY9FO4WffKUiw==,103.253.105.84,1781,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,start_listening,AQMFcbsrH1bpq3DQf1dyuWGHzgAAA+g=,client,342346882800025600,Tns,AABCE61CwASZPnrzTA4TTYEBAADoAwAA,analytics-ingest-prd-q0pz,HiRxikGPr_LjE-4WQBUZTw==,103.253.105.84,1772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Analysis
